In [ ]:
#!pip install wandb
#import wandb
#wandb.init()

In [ ]:
import os
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow as tf
from tensorflow import keras


import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model

from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

In [ ]:
%cd 'My Drive'

%cd 'Action Recognition'

In [ ]:
image_height, image_width = 64, 64
images_per_class = 8000
dataset_directory = "hmdb51"
classes_list = ["pullup", "punch", "dive", "fencing", "ride_bike", "golf"]
model_output_size = len(classes_list)

In [ ]:
def frames_extraction(video_path):
    frames_list = []
    video_reader = cv2.VideoCapture(video_path)

  
    while True:
        success, frame = video_reader.read() 
        if not success:
            print("Defected frame")
            break
        resized_frame = cv2.resize(frame, (image_height, image_width))
        normalized_frame = preprocess_input(resized_frame)
        frames_list.append(normalized_frame)
    
    video_reader.release()

    return frames_list

In [ ]:
def create_dataset():

    
    temp_features = [] 
    features = []
    labels = []
    
  
    for class_index, class_name in enumerate(classes_list):
        print(f'Extracting Data of Class: {class_name}')
        
       
        files_list = os.listdir(os.path.join(dataset_directory, class_name))

      
        for file_name in files_list:
            video_file_path = os.path.join(dataset_directory, class_name, file_name)
            frames = frames_extraction(video_file_path)
            temp_features.extend(frames)
        features.extend(random.sample(temp_features, images_per_class))
        labels.extend([class_index] * images_per_class)
        temp_features.clear()

    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels

In [ ]:
features, labels = create_dataset()

In [ ]:
seed_constant = 23
np.random.seed(seed_constant)
random.seed(seed_constant)
tf.random.set_seed(seed_constant)

In [ ]:
print (features.shape)
print (labels.shape)

one_hot_encoded_labels = to_categorical(labels)

features_train, features_test, labels_train, labels_test = train_test_split(features, one_hot_encoded_labels, test_size = 0.2, shuffle = True, random_state = seed_constant)

print (features_train.shape)
print (labels_train.shape)


In [ ]:
print (labels_train)

In [ ]:
# load model
base_model = Sequential()
base_model.add(VGG16(input_shape=(64,64,3), weights='imagenet', include_top=False, pooling='avg'))
base_model.add(Dense(288, activation = 'relu'))
base_model.add(Dense(288, activation = 'relu'))
base_model.add(Dense(6, activation='softmax'))
# summarize the model

base_model.layers[0].trainable = False




In [ ]:
base_model.compile(optimizer='sgd',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
plot_model(base_model,show_shapes = True, show_layer_names = True)

In [ ]:
# Adding Early Stopping Callback
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, mode = 'min', restore_best_weights = True)

# Start Training
model_training_history = base_model.fit(x = features_train, y = labels_train, epochs = 40, batch_size = 16 , shuffle = True, validation_split = 0.2, callbacks = [early_stopping_callback])

In [ ]:
model_evaluation_history = base_model.evaluate(features_test, labels_test)

from sklearn.metrics import classification_report

y_pred = base_model.predict(features_test, batch_size=4, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)

l_test=np.argmax(labels_test, axis=1)

print(classification_report(l_test, y_pred_bool))

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(l_test, y_pred_bool)

print (cm)

In [ ]:
def plot_metric(metric_name_1, metric_name_2, plot_name):
  # Get Metric values using metric names as identifiers
  metric_value_1 = model_training_history.history[metric_name_1]
  metric_value_2 = model_training_history.history[metric_name_2]

  # Constructing a range object which will be used as time 
  epochs = range(len(metric_value_1))
  
  # Plotting the Graph
  plt.plot(epochs, metric_value_1, 'blue', label = metric_name_1)
  plt.plot(epochs, metric_value_2, 'red', label = metric_name_2)
  
  # Adding title to the plot
  plt.title(str(plot_name))

  # Adding legend to the plot
  plt.legend()

In [ ]:
plot_metric('loss', 'val_loss', 'Total Loss vs Total Validation Loss')

In [ ]:
plot_metric('accuracy', 'val_accuracy', 'Total Accuracy vs Total Validation Accuracy')

In [ ]:
from collections import Counter

def get_first_mode(a):
    c = Counter(a)  
    mode_count = max(c.values())
    mode = {key for key, count in c.items() if count == mode_count}
    first_mode = next(x for x in a if x in mode)
    return first_mode

In [ ]:


def frames_extraction2(video_path):
    frames_list = []
    
    vidObj = cv2.VideoCapture(video_path)
    skip_frames=30
  


    # Used as counter variable 
    count = 0
 
    while True: 
         
        success, image = vidObj.read() 

        if success == False:
          print("Defected frame")
          break

        if count == 0:
            image = cv2.resize(image, (image_height, image_width))
            normalized_image = preprocess_input(image)
            frames_list.append(normalized_image)
        
        else:
          if count % 25 == 0:
            image = cv2.resize(image, (image_height, image_width))
            normalized_image = preprocess_input(image)
            frames_list.append(normalized_image)
        
        count += 1
            
       
            
    return frames_list

In [ ]:
#Evaluating a different dataset

from tqdm import tqdm
from statistics import mode


predict = []
actual = []
dataset_directory2="UCF50"

temp_features = [] 
features = []
labels = []

cc=0

for class_index, class_name in enumerate(classes_list):
    print(f'Extracting Data of Class: {class_name}')
    
    files_list = os.listdir(os.path.join(dataset_directory2, class_name))

    for file_name in files_list:

        video_file_path = os.path.join(dataset_directory2, class_name, file_name)

        frames = frames_extraction2(video_file_path)

        temppred=[]

        for i in frames:
          temppred.append(base_model.predict_classes(np.expand_dims(i, axis = 0))[0])
        
        print (temppred)
        print ("mode", get_first_mode(temppred), cc)
        cc+=1
        predict.append(get_first_mode(temppred))
        actual.append(class_index)





In [ ]:
print(classification_report(actual, predict))


In [ ]:
print(confusion_matrix(actual, predict))